In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ============================================================================
# STEP 1: Load PIOP-2 (Resting-State) and Train Model
# ============================================================================
print("="*70)
print("STEP 1: Training Model on PIOP-2 (Resting-State Data)")
print("="*70)

# Load PIOP-2
df_piop2 = pd.read_csv('PIOP2_restingstate.csv')
print(f"PIOP-2 shape: {df_piop2.shape}")

# Extract connection columns and subjects
connection_columns = df_piop2.columns[1:].tolist()
subjects_piop2 = df_piop2.iloc[:, 0].values

# Extract unique regions
unique_regions = set()
for col in connection_columns:
    regions = col.split('~')
    if len(regions) == 2:
        unique_regions.add(regions[0])
        unique_regions.add(regions[1])

region_list = sorted(list(unique_regions))
n_regions = len(region_list)
region_to_idx = {region: idx for idx, region in enumerate(region_list)}

print(f"Number of brain regions: {n_regions}")

# Function to reconstruct connectivity matrix
def reconstruct_connectivity_matrix(row_data, connection_columns, region_to_idx, n_regions):
    matrix = np.zeros((n_regions, n_regions))
    for col_name, value in zip(connection_columns, row_data):
        regions = col_name.split('~')
        if len(regions) == 2:
            idx1 = region_to_idx[regions[0]]
            idx2 = region_to_idx[regions[1]]
            matrix[idx1, idx2] = value
            matrix[idx2, idx1] = value
    np.fill_diagonal(matrix, 1.0)
    return matrix

# Create PIOP-2 dataset
print("\nCreating PIOP-2 training dataset...")
X_piop2_list = []
y_piop2_list = []

for subject_idx in range(df_piop2.shape[0]):
    connectivity_values = df_piop2.iloc[subject_idx, 1:].values
    conn_matrix = reconstruct_connectivity_matrix(
        connectivity_values, connection_columns, region_to_idx, n_regions
    )
    
    for region_idx in range(n_regions):
        connectivity_pattern = conn_matrix[region_idx, :].copy()
        connectivity_pattern = np.delete(connectivity_pattern, region_idx)  # Remove diagonal
        X_piop2_list.append(connectivity_pattern)
        y_piop2_list.append(region_idx)

X_piop2 = np.array(X_piop2_list)
y_piop2 = np.array(y_piop2_list)

print(f"PIOP-2 dataset: {X_piop2.shape[0]} samples, {X_piop2.shape[1]} features")

# Train model on ALL PIOP-2 data
print("\nTraining logistic regression on entire PIOP-2 dataset...")
scaler = StandardScaler()
X_piop2_scaled = scaler.fit_transform(X_piop2)

model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    C=0.1
)

model.fit(X_piop2_scaled, y_piop2)

# Evaluate on training data
y_piop2_pred = model.predict(X_piop2_scaled)
piop2_accuracy = accuracy_score(y_piop2, y_piop2_pred)
print(f"PIOP-2 training accuracy: {piop2_accuracy:.4f}")


In [ ]:
# ============================================================================
# STEP 2: Load PIOP-1 (Task-Based) Data
# ============================================================================
print("\n" + "="*70)
print("STEP 2: Loading PIOP-1 (Task-Based Gender Task Data)")
print("="*70)

df_piop1 = pd.read_csv('PIOP1_gstroop.csv')
print(f"PIOP-1 shape: {df_piop1.shape}")

# Verify columns match
assert df_piop1.columns[1:].tolist() == connection_columns, "Column mismatch between PIOP-1 and PIOP-2!"
print("✓ Column structure matches PIOP-2")

subjects_piop1 = df_piop1.iloc[:, 0].values
print(f"Number of PIOP-1 subjects: {len(subjects_piop1)}")

In [ ]:

# ============================================================================
# STEP 3: Apply Model to PIOP-1 Data
# ============================================================================
print("\n" + "="*70)
print("STEP 3: Applying Resting-State Model to Task Data")
print("="*70)

# Create PIOP-1 dataset
print("Creating PIOP-1 dataset...")
X_piop1_list = []
y_piop1_list = []
subject_piop1_list = []

for subject_idx in range(df_piop1.shape[0]):
    connectivity_values = df_piop1.iloc[subject_idx, 1:].values
    conn_matrix = reconstruct_connectivity_matrix(
        connectivity_values, connection_columns, region_to_idx, n_regions
    )
    
    for region_idx in range(n_regions):
        connectivity_pattern = conn_matrix[region_idx, :].copy()
        connectivity_pattern = np.delete(connectivity_pattern, region_idx)
        X_piop1_list.append(connectivity_pattern)
        y_piop1_list.append(region_idx)
        subject_piop1_list.append(subjects_piop1[subject_idx])

X_piop1 = np.array(X_piop1_list)
y_piop1 = np.array(y_piop1_list)
subjects_piop1_array = np.array(subject_piop1_list)

print(f"PIOP-1 dataset: {X_piop1.shape[0]} samples, {X_piop1.shape[1]} features")

# Apply scaling and predict
X_piop1_scaled = scaler.transform(X_piop1)
y_piop1_pred = model.predict(X_piop1_scaled)

# Get prediction probabilities
y_piop1_proba = model.predict_proba(X_piop1_scaled)

In [ ]:

# ============================================================================
# STEP 4: Create Error Maps
# ============================================================================
print("\n" + "="*70)
print("STEP 4: Creating Error Maps - Detecting Altered Connectivity")
print("="*70)

# Overall accuracy on PIOP-1
piop1_accuracy = accuracy_score(y_piop1, y_piop1_pred)
print(f"\nOverall Classification Performance:")
print(f"  PIOP-2 (Resting-State): {piop2_accuracy:.4f}")
print(f"  PIOP-1 (Gender Task):   {piop1_accuracy:.4f}")
print(f"  Accuracy Drop:          {piop2_accuracy - piop1_accuracy:.4f}")

# Per-region error rates
cm_piop1 = confusion_matrix(y_piop1, y_piop1_pred)
per_region_accuracy_piop1 = np.diag(cm_piop1) / cm_piop1.sum(axis=1)

cm_piop2 = confusion_matrix(y_piop2, y_piop2_pred)
per_region_accuracy_piop2 = np.diag(cm_piop2) / cm_piop2.sum(axis=1)

# Calculate error increase per region
error_rate_piop2 = 1 - per_region_accuracy_piop2
error_rate_piop1 = 1 - per_region_accuracy_piop1
error_increase = error_rate_piop1 - error_rate_piop2

print(f"\nPer-Region Error Analysis:")
print(f"  Mean error increase: {error_increase.mean():.4f}")
print(f"  Std error increase:  {error_increase.std():.4f}")
print(f"  Regions with increased error: {(error_increase > 0).sum()} / {n_regions}")

In [ ]:
# ============================================================================
# STEP 5: Identify Regions with Altered Connectivity
# ============================================================================
print("\n" + "="*70)
print("STEP 5: Regions with Significantly Altered Connectivity")
print("="*70)

# Statistical test: Compare error rates
# Regions where task connectivity significantly differs from resting-state

# Calculate confidence intervals
alpha = 0.05
n_samples_per_region = X_piop1.shape[0] // n_regions

# Identify top regions with increased errors
top_altered_regions = np.argsort(error_increase)[-20:][::-1]

print(f"\nTop 20 Regions with INCREASED Errors (Altered Connectivity):")
print(f"{'Rank':<5} {'Region':<50} {'RS Error':<12} {'Task Error':<12} {'Increase':<10}")
print("-" * 95)
for rank, region_idx in enumerate(top_altered_regions, 1):
    region_name = region_list[region_idx]
    print(f"{rank:<5} {region_name:<50} {error_rate_piop2[region_idx]:.4f}       "
          f"{error_rate_piop1[region_idx]:.4f}        {error_increase[region_idx]:+.4f}")

# Identify regions with decreased errors (more stable during task)
top_stable_regions = np.argsort(error_increase)[:20]

print(f"\nTop 20 Regions with DECREASED Errors (More Stable/Task-Relevant):")
print(f"{'Rank':<5} {'Region':<50} {'RS Error':<12} {'Task Error':<12} {'Change':<10}")
print("-" * 95)
for rank, region_idx in enumerate(top_stable_regions, 1):
    region_name = region_list[region_idx]
    print(f"{rank:<5} {region_name:<50} {error_rate_piop2[region_idx]:.4f}       "
          f"{error_rate_piop1[region_idx]:.4f}        {error_increase[region_idx]:+.4f}")


In [ ]:
# ============================================================================
# STEP 6: Prediction Confidence Analysis
print("\n" + "="*70)
print("STEP 6: Prediction Confidence Analysis")
print("="*70)

# Average prediction confidence (max probability)
confidence_piop1 = y_piop1_proba.max(axis=1)

# Group by region
confidence_by_region = np.zeros(n_regions)
for region_idx in range(n_regions):
    region_mask = y_piop1 == region_idx
    confidence_by_region[region_idx] = confidence_piop1[region_mask].mean()

print(f"\nPrediction Confidence Statistics:")
print(f"  Mean confidence: {confidence_piop1.mean():.4f}")
print(f"  Std confidence:  {confidence_piop1.std():.4f}")

# Low confidence regions = uncertain/altered connectivity
low_confidence_regions = np.argsort(confidence_by_region)[:15]

print(f"\nTop 15 Regions with LOWEST Prediction Confidence:")
print(f"{'Rank':<5} {'Region':<50} {'Confidence':<12}")
print("-" * 70)
for rank, region_idx in enumerate(low_confidence_regions, 1):
    region_name = region_list[region_idx]
    print(f"{rank:<5} {region_name:<50} {confidence_by_region[region_idx]:.4f}")

In [ ]:

# ============================================================================
# STEP 7: Visualizations
# ============================================================================
print("\n" + "="*70)
print("STEP 7: Generating Comprehensive Visualizations")
print("="*70)

fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.35)

# Plot 1: Error rate comparison
ax1 = fig.add_subplot(gs[0, :2])
x = np.arange(n_regions)
ax1.scatter(error_rate_piop2, error_rate_piop1, alpha=0.5, s=30)
ax1.plot([0, 1], [0, 1], 'r--', label='Equal Error', linewidth=2)
ax1.set_xlabel('Resting-State Error Rate', fontsize=12)
ax1.set_ylabel('Task Error Rate', fontsize=12)
ax1.set_title('Error Rate Comparison: Resting-State vs Task', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Add annotations for extreme points
for region_idx in top_altered_regions[:3]:
    ax1.annotate(region_list[region_idx].split('_')[-1][:10], 
                 (error_rate_piop2[region_idx], error_rate_piop1[region_idx]),
                 fontsize=8, alpha=0.7)

# Plot 2: Error increase distribution
ax2 = fig.add_subplot(gs[0, 2])
ax2.hist(error_increase, bins=40, alpha=0.7, edgecolor='black')
ax2.axvline(0, color='r', linestyle='--', linewidth=2, label='No change')
ax2.axvline(error_increase.mean(), color='g', linestyle='--', linewidth=2, 
            label=f'Mean: {error_increase.mean():.3f}')
ax2.set_xlabel('Error Increase (Task - Rest)', fontsize=11)
ax2.set_ylabel('Number of Regions', fontsize=11)
ax2.set_title('Distribution of Error Changes', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(axis='y', alpha=0.3)

# Plot 3: Top altered regions
ax3 = fig.add_subplot(gs[1, :])
top_20_idx = top_altered_regions[:20]
region_labels = [region_list[i].replace('LH_', '').replace('RH_', '')[:30] for i in top_20_idx]
colors_altered = ['red' if error_increase[i] > 0.1 else 'orange' for i in top_20_idx]

ax3.barh(range(20), error_increase[top_20_idx], color=colors_altered, alpha=0.7, edgecolor='black')
ax3.set_yticks(range(20))
ax3.set_yticklabels(region_labels, fontsize=9)
ax3.set_xlabel('Error Increase (Task - Resting)', fontsize=12)
ax3.set_title('Top 20 Regions with Altered Connectivity During Gender Task', 
              fontsize=13, fontweight='bold')
ax3.axvline(0, color='black', linestyle='-', linewidth=1)
ax3.grid(axis='x', alpha=0.3)

# Plot 4: Confidence by error increase
ax4 = fig.add_subplot(gs[2, 0])
ax4.scatter(error_increase, confidence_by_region, alpha=0.6, s=30)
ax4.set_xlabel('Error Increase', fontsize=11)
ax4.set_ylabel('Mean Prediction Confidence', fontsize=11)
ax4.set_title('Confidence vs Error Change', fontsize=12, fontweight='bold')
ax4.grid(alpha=0.3)

# Plot 5: Per-region accuracy comparison
ax5 = fig.add_subplot(gs[2, 1])
ax5.bar(range(n_regions), per_region_accuracy_piop2, alpha=0.5, label='Resting-State', color='blue')
ax5.bar(range(n_regions), per_region_accuracy_piop1, alpha=0.5, label='Task', color='red')
ax5.set_xlabel('Region Index', fontsize=11)
ax5.set_ylabel('Classification Accuracy', fontsize=11)
ax5.set_title('Per-Region Accuracy Comparison', fontsize=12, fontweight='bold')
ax5.legend()
ax5.grid(axis='y', alpha=0.3)

# Plot 6: Summary statistics
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')
summary_text = f"""
SUMMARY STATISTICS

Dataset Sizes:
  PIOP-2 (Rest): {df_piop2.shape[0]} subjects
  PIOP-1 (Task): {df_piop1.shape[0]} subjects
  Brain Regions: {n_regions}

Overall Accuracy:
  Resting-State: {piop2_accuracy:.2%}
  Gender Task:   {piop1_accuracy:.2%}
  Drop:          {(piop2_accuracy-piop1_accuracy):.2%}

Error Changes:
  Mean increase: {error_increase.mean():.4f}
  Regions worse: {(error_increase > 0.05).sum()}
  Regions better: {(error_increase < -0.05).sum()}

Confidence:
  Mean: {confidence_piop1.mean():.3f}
  Min:  {confidence_piop1.min():.3f}
  Max:  {confidence_piop1.max():.3f}
"""
ax6.text(0.1, 0.5, summary_text, fontsize=10, family='monospace',
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.savefig('piop1_task_connectivity_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Saved: piop1_task_connectivity_analysis.png")

print("\n" + "="*70)
print("ANALYSIS COMPLETE!")
print("="*70)
print(f"\n🎯 Key Findings:")
print(f"  • Accuracy dropped by {(piop2_accuracy-piop1_accuracy)*100:.1f}% during gender task")
print(f"  • {(error_increase > 0.05).sum()} regions show notably altered connectivity")
print(f"  • {(error_increase < -0.05).sum()} regions became MORE identifiable during task")
print(f"  • Top altered regions likely involved in gender processing/task execution")
print(f"\n📊 Next steps:")
print(f"  1. Investigate anatomical locations of top altered regions")
print(f"  2. Compare with known gender processing networks")
print(f"  3. Analyze which functional networks are most affected")
print(f"  4. Consider subject-level variability in connectivity changes")